In [ ]:
!pip install langchain
!pip install langchain_experimental
!pip install openai
!pip install openai google-search-results
!pip install Wikipedia

In [1]:
import json
import os
f = open('keys.json')
keys = json.load(f)
OPENAI_API_KEY = keys["openai_api_key"]
SERP_API_KEY = keys["serpapi_api_key"]


os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["SERPAPI_API_KEY"] = SERP_API_KEY

# [OpenAI function](https://python.langchain.com/docs/modules/agents/agent_types/openai_functions_agent)

In [25]:
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.utilities import SQLDatabase

In [28]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="FooBar-DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context"
    )
]

In [29]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [30]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI assistant.\nHuman: Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [2.30s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "name": "Search",
       

"Leo DiCaprio's girlfriend is Blake Lively. Her current age raised to the 0.43 power is approximately 4.19."

# [OpenAI Multi Functions](https://python.langchain.com/docs/modules/agents/agent_types/openai_multi_functions_agent)

In [19]:
from langchain import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
# Do this so we can see exactly what's going on under the hood
import langchain

langchain.debug = True

In [21]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
search = SerpAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful when you need to answer questions about current events. You should ask targeted questions.",
    ),
]

In [22]:
mrkl.run("What is the weather in LA and SF?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the weather in LA and SF?"
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI assistant.\nHuman: What is the weather in LA and SF?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [3.28s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "name": "tool_selection",
                "arguments": "{\n  \"actions\": [\n    {\n      \"action_name\": \"Search\",\n     

'The weather in Los Angeles is currently cloudy with a low of 68°F. The weather in San Francisco is currently partly cloudy with a high of 72°F.'

mrkl = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_MULTI_FUNCTIONS, verbose=True
)

# Plan and execute

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain

In [11]:
search = SerpAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

def string_length(query: str) -> str:
    """Counts the number of characters in a string."""
    return str(len(query))


tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool.from_function(
        func= lambda str : len(str),
        name="String Length",
        description="Counts the number of characters in a string",
    )
]

In [13]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)
agent.run("How many letters in the capital of France?")



> Entering new PlanAndExecute chain...
steps=[Step(value='Look up the capital of France.'), Step(value='Count the number of letters in the capital.'), Step(value="Given the above steps taken, respond to the user's original question.\n\n")]

> Entering new AgentExecutor chain...
Thought: To find the capital of France, I can use the Search tool to look it up online.

Action:
```{"action": "Search", "action_input": "capital of France"}```

> Finished chain.
*****

Step: Look up the capital of France.

Response: Thought: To find the capital of France, I can use the Search tool to look it up online.

Action:
```{"action": "Search", "action_input": "capital of France"}```

> Entering new AgentExecutor chain...
Thought: To count the number of letters in the capital of France, I can use the String Length tool. 

Action:
```{"action": "String Length", "action_input": "Paris"}```

> Finished chain.
*****

Step: Count the number of letters in the capital.

Response: Thought: To count the number

'Action:\n```{"action": "Final Answer", "action_input": "The capital of France is Paris."}```'

# [ReAct](https://python.langchain.com/docs/modules/agents/agent_types/react)

In [2]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [3]:
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [4]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Blake Lively and DiCaprio are believed to have enjoyed a whirlwind five-month romance in 2011. The pair were seen on a yacht together in Cannes, ...
Thought: I need to find out how old Blake Lively is.
Action: Search
Action Input: "Blake Lively age"
Observation: 36 years
Thought: I need to calculate 36 raised to the 0.43 power.
Action: Calculator
Action Input: 36^0.43
Observation: Answer: 4.6688516567750975
Thought: I now know the final answer.
Final Answer: Blake Lively, Leo DiCaprio's girlfriend, is 36 years old and her age raised to the 0.43 power is 4.6688516567750975.

> Finished chain.


"Blake Lively, Leo DiCaprio's girlfriend, is 36 years old and her age raised to the 0.43 power is 4.6688516567750975."

In [5]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0)
agent = initialize_agent(tools, chat_model, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
Thought: I need to search for information about Leo DiCaprio's girlfriend and then calculate her current age raised to the 0.43 power.

Action:
```
{
  "action": "Search",
  "action_input": "Leo DiCaprio girlfriend"
}
```


Observation: Blake Lively and DiCaprio are believed to have enjoyed a whirlwind five-month romance in 2011. The pair were seen on a yacht together in Cannes, ...
Thought:I need to search again to find Leo DiCaprio's current girlfriend. Then I can calculate her age raised to the 0.43 power.

Action:
```
{
  "action": "Search",
  "action_input": "Leo DiCaprio current girlfriend"
}
```
Observation: Leonardo DiCaprio has split from girlfriend Camila Morrone. Getty. Taking a look at his dating history, Leo has always had a preference for ...
Thought:I now know that Leo DiCaprio's current girlfriend is Camila Morrone. Now I can calculate her age raised to the 0.43 power.

Action:
```
{
  "action": "Calculator",
  "action_input": "22

"Camila Morrone's current age raised to the 0.43 power is approximately 3.7778."

# [ReAct document store](https://python.langchain.com/docs/modules/agents/agent_types/react_docstore)

In [16]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
]

llm = OpenAI(temperature=0, model_name="text-davinci-002", openai_api_key=OPENAI_API_KEY)
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [33]:
question = "When did the first British come to Australia?"
react.run(question)



> Entering new AgentExecutor chain...

Thought: I need to search Australia and find when the first British came.

Action: Search[Australia]

Observation: Australia, officially the Commonwealth of Australia, is a sovereign country comprising the mainland of the Australian continent, the island of Tasmania, and numerous smaller islands. Australia is the largest country by area in Oceania and the world's sixth-largest country. Australia is the oldest, flattest, and driest inhabited continent, with the least fertile soils. It is a megadiverse country, and its size gives it a wide variety of landscapes and climates, with deserts in the centre, tropical rainforests in the north-east, tropical savannas in the north, and mountain ranges in the south-east.
The ancestors of Aboriginal Australians began arriving from south-east Asia approximately 65,000 years ago, during the last ice age. Arriving by sea, they settled the continent and had formed approximately 250 distinct language groups by th

OutputParserException: ignored

In [29]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0)
agent = initialize_agent(tools, chat_model, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
Thought: I need to find out who is Leo DiCaprio's girlfriend first. Then I can calculate her current age raised to the 0.43 power.

Action:
```
{
  "action": "Search",
  "action_input": "Leo DiCaprio girlfriend"
}
```



ValueError: ignored